# Multi-robot Task Allocation

In [129]:
using Symbolics
using Test
using Plots
using Latexify
using LaTeXStrings
using ForwardDiff: jacobian, gradient, derivative
using LinearAlgebra
using SparseArrays

In [109]:
function forward_pose(r)
    x₁, y₁, θ₁, x₂, y₂, θ₂ = r

    a = x₁ - x₂
    b = y₁ - y₂

    xᶜ = (x₁ + x₂) / 2
    yᶜ = (y₁ + y₂) / 2
    d = (1 / 2) * sqrt(a^2 + b^2)

    θᶜ = atan(a,b)

    ϕ₁ = θ₁ - θᶜ
    ϕ₂ = θ₂ - θᶜ
    
    return xᶜ, yᶜ, d, θᶜ, ϕ₁, ϕ₂
end

function inverse_pose(c)
    xᶜ, yᶜ, d, θᶜ, ϕ₁, ϕ₂ = c

    x₁ = xᶜ + d * sin(θᶜ)
    x₂ = xᶜ - d * sin(θᶜ)

    y₁ = yᶜ + d * cos(θᶜ)
    y₂ = yᶜ - d * cos(θᶜ)

    θ₁ = θᶜ + ϕ₁
    θ₂ = θᶜ + ϕ₂

    return x₁, y₁, θ₁, x₂, y₂, θ₂
end;

In [110]:
function test_forward_pose()
    robots_pose = [3.5, -2.2, 1.0, 0.0, 3.4, 33.3]

    cluster_pose = forward_pose(robots_pose)

    @test all(isapprox.(cluster_pose, [1.75, 0.6, 3.3018, 2.5829, -1.5829, 30.7170], atol=1e-3))
end

function test_inverse_pose()
    cluster_pose = [1.75, 0.6, 3.3018, 2.5829, -1.5829, 30.7170]
    robots_pose = inverse_pose(cluster_pose)

    @test all(isapprox.(robots_pose, [3.5, -2.2, 1.0, 0.0, 3.4, 33.3], atol=1e-3))
end

test_forward_pose()
test_inverse_pose()

Test Passed

In [111]:
num_robots = 1  # Number of robots
num_tasks = 3  # Number of tasks
DOF = 3  # Number of Degrees of Freedom (DOF)
control_dim = 3  # Control vector dimension

3

In [112]:
T = 100
t = 0:T/100:100

Izz = 1
m = 10
bx = 5
by = 5
bθ = 1

goals = [
    10 10 3*π/2;
    15 10 3*π/2;
    10 15 3*π/2
]

# A, B = ...
# ss = ...

r = zeros(T, 2 * DOF);
rdot = zeros(T, 2 * DOF);

r[1,:] = [5 1 0 15 1 0]

c = zeros(T, 2 * DOF);
cdot = zeros(T, 2 * DOF);

$$\vec{R} = [x_1, y_1, \theta_1, x_2, y_2, \theta_2]^T$$
$$\vec{C} = [x_c, y_c, d,\theta_c, \phi_1, \phi_2]^T$$

In [113]:
function forward_kin_jacobian(robots_pose)
    """
    Compute jacobian matrix based on the forward kinematics relationship
    
    Arguments:
        robots_pose: robots position vector defined as (x₁, y₁, θ₁, x₂, y₂, θ₂)
    Returns:
        J: jacobian matrix
    """
    f(x) = [
        (x[1] + x[4]) / 2;
        (x[2] + x[5]) / 2;
        1 / 2 * sqrt((x[1] - x[4])^2+(x[2] - x[5])^2);
        atan(x[2] - x[5], x[1] - x[4]);
        x[3] - atan(x[2] - x[5], x[1] - x[4]);
        x[6] - atan(x[2] - x[5], x[1] - x[4]);
    ]

    return jacobian(f, robots_pose)    
end;

In [114]:
function test_forward_kin_jacobian()
    robots_pose = [3.5, -2.2, 1.0, 0, 3.4, 33.3]
    J = forward_kin_jacobian(robots_pose)

    expected = [
        0.5 0 0 0.5 0 0;
        0 0.5 0 0 0.5 0;
        0.26499947 -0.42399915 0 -0.26499947 0.42399915 0;
        0.12841091 0.08025682 0 -0.12841091 -0.08025682 0;
        -0.12841091 -0.08025682 1 0.12841091 0.08025682 0;
        -0.12841091 -0.08025682 0 0.12841091 0.08025682 1
    ]
    
    @test all(isapprox.(J, expected, atol=1e-3))
end

test_forward_kin_jacobian()

Test Passed

In [115]:
function inverse_kin_jacobian(cluster_pose)
    """
    Compute jacobian matrix based on the inverse kinematics relationship

    Arguments:
        cluster_pose: cluster position vector defined as (x_c, y_c, d, θ, φ₁, φ₂)
    Returns:
        J⁻¹: jacobian matrix
    """
    f⁻¹(x) = [
        x[1] + x[3] * sin(x[4]);
        x[2] + x[3] * cos(x[4]);
        x[4] + x[5];
        x[1] - x[3] * sin(x[4]);
        x[2] - x[3] * cos(x[4]);
        x[4] + x[6];
    ]

    return jacobian(f⁻¹, cluster_pose)    
end;

In [116]:
function test_inverse_kin_jacobian()
    cluster_pose = [1.8, 0.6, 3.3, 2.6, -1.6, 30.7]
    
    J⁻¹ = inverse_kin_jacobian(cluster_pose)
    
    expected = [
        1 0 0.51550137 -2.82773289 0 0;
        0 1 -0.85688875 -1.70115453 0 0;
        0 0 0 1 1 0;
        1 0 -0.51550137 2.82773289 0 0;
        0 1 0.85688875 1.70115453 0 0;
        0 0 0 1 0 1
    ]
    
    @test all(isapprox.(J⁻¹, expected, atol=1e-3))
end

test_inverse_kin_jacobian()

Test Passed

In [117]:
function dot_forward_jacobian(r, dr)
    """
    Compute derivative of the jacobian matrix (forward kinematics relationship) with respect to time t

    Arguments:
        r: position vector of the robots defined as (x₁, y₁, θ₁, x₂, y₂, θ₂)
        dr: velocities vector of the robots defined as (dx₁, dy₁, dθ₁, dx₂, dy₂, dθ₂)
    Returns:
        Jₜ: jacobian matrix derivative with respect to time t for a given robot positions and velocities
    """
    
    x₁, y₁, _, x₂, y₂, _ = r
    dx₁, dy₁, _, dx₂, dy₂, _ = dr
    
    B = (x₁ - x₂) ^ 2 + (y₁ - y₂) ^ 2
    
    Jₜ = zeros(6, 6)
    
    Jₜ[3, :] = 1 / (2 * B^(3 / 2)) * [
        (y₁ - y₂) * ((y₁ - y₂) * (dx₁ - dx₂) - (x₁ - x₂) * (dy₁ - dy₂)),
        (x₁ - x₂) * (-(y₁ - y₂) * (dx₁ - dx₂) + (x₁ - x₂) * (dy₁ - dy₂)),
        0,
        (y₁ - y₂) * (-(y₁ - y₂) * (dx₁ - dx₂) + (x₁ - x₂) * (dy₁ - dy₂)),
        -(x₁ - x₂) * (-(y₁ - y₂) * (dx₁ - dx₂) + (x₁ - x₂) * (dy₁ - dy₂)),
        0
    ]
    
    Jₜ[4, :] = (1 / B^2) * [
        2 * (x₁ - x₂) * (y₁ - y₂) * dx₁ - 2 * (x₁ - x₂) * (y₁ - y₂) * dx₂ - (x₁ - x₂ + y₁ - y₂) * (x₁ - x₂ - y₁ + y₂) *
        (dy₁ - dy₂),
        B * (dx₁ - dx₂) - (x₁ - x₂) * (2 * (x₁ - x₂) * (dx₁ - dx₂) + 2 * (y₁ - y₂) * (dy₁ - dy₂)),
        0,
        -2 * (x₁ - x₂) * (y₁ - y₂) * dx₁ + 2 * (x₁ - x₂) * (y₁ - y₂) * dx₂ + (x₁ - x₂ + y₁ - y₂) * (x₁ - x₂ - y₁ + y₂) *
        (dy₁ - dy₂),
        B * (-dx₁ + dx₂) - (-x₁ + x₂) * (2 * (x₁ - x₂) * (dx₁ - dx₂) + 2 * (y₁ - y₂) * (dy₁ - dy₂)),
        0
    ]
    
    Jₜ[5, :] = (1 / B^2) * [
        -2 * (x₁ - x₂) * (y₁ - y₂) * dx₁ + 2 * (x₁ - x₂) * (y₁ - y₂) * dx₂ + (x₁ - x₂ + y₁ - y₂) * (x₁ - x₂ - y₁ + y₂) *
        (dy₁ - dy₂),
        B * (-dx₁ + dx₂) - (-x₁ + x₂) * (2 * (x₁ - x₂) * (dx₁ - dx₂) + 2 * (y₁ - y₂) * (dy₁ - dy₂)),
        0,
        2 * (x₁ - x₂) * (y₁ - y₂) * dx₁ - 2 * (x₁ - x₂) * (y₁ - y₂) * dx₂ - (x₁ - x₂ + y₁ - y₂) * (x₁ - x₂ - y₁ + y₂) *
        (dy₁ - dy₂),
        B * (dx₁ - dx₂) - (x₁ - x₂) * (2 * (x₁ - x₂) * (dx₁ - dx₂) + 2 * (y₁ - y₂) * (dy₁ - dy₂)),
        0
    ]

    Jₜ[6, :] = (1 / B^2) * [
        -2 * (x₁ - x₂) * (y₁ - y₂) * dx₁ + 2 * (x₁ - x₂) * (y₁ - y₂) * dx₂ + (x₁ - x₂ + y₁ - y₂) * (x₁ - x₂ - y₁ + y₂) *
        (dy₁ - dy₂),
        B * (-dx₁ + dx₂) - (-x₁ + x₂) * (2 * (x₁ - x₂) * (dx₁ - dx₂) + 2 * (y₁ - y₂) * (dy₁ - dy₂)),
        0,
        2 * (x₁ - x₂) * (y₁ - y₂) * dx₁ - 2 * (x₁ - x₂) * (y₁ - y₂) * dx₂ - (x₁ - x₂ + y₁ - y₂) * (x₁ - x₂ - y₁ + y₂) *
        (dy₁ - dy₂),
        B * (dx₁ - dx₂) - (x₁ - x₂) * (2 * (x₁ - x₂) * (dx₁ - dx₂) + 2 * (y₁ - y₂) * (dy₁ - dy₂)),
        0
    ]
    
    return Jₜ
end;

In [118]:
function test_dot_forward_jacobian()
    robots_pose = [3.5, -2.2, 1.0, 0, 3.4, 33.3]
    robots_velocities = [-1.2, 2, 1.0, 2.2, -3.4, -1.0]
    
    Jₜ = dot_forward_jacobian(robots_pose, robots_velocities)
    
    expected = [
        0 0 0 0 0 0;
        0 0 0 0 0 0;
        -0.00136115 -0.00085072 0 0.00136115 0.00085072 0;
        0.1243401 0.0771393 0 -0.1243401 -0.0771393 0;
        -0.1243401 -0.0771393 0 0.1243401 0.0771393 0;
        -0.1243401 -0.0771393 0 0.1243401 0.0771393 0
    ]
    
    @test all(isapprox.(Jₜ, expected, atol=1e-3))
end

test_dot_forward_jacobian()

Test Passed

In [140]:
function cluster_matrices(m, Izz, bx, by, bθ)
    Aᵢ = sparse(diagm([m, m, Izz]))
    Bᵢ = sparse(diagm([bx, by, bθ]))
       
    return blockdiag(Aᵢ, Aᵢ), blockdiag(Bᵢ, Bᵢ)
end;

In [142]:
function test_cluster_matrices()
    Izz = 1
    m = 10
    bx = 5
    by = 5
    bθ = 1

    A, B = cluster_matrices(m, Izz, bx, by, bθ)
    
    Aᵢ = sparse(diagm([10, 10, 1]))
    Bᵢ = sparse(diagm([5, 5, 1]))

    @test all(isapprox.(A, blockdiag(Aᵢ, Aᵢ), atol=1e-3))
    @test all(isapprox.(B, blockdiag(Bᵢ, Bᵢ), atol=1e-3))
end

test_cluster_matrices()

Test Passed

In [149]:
function cluster_state_space(m, Izz, bx, by, bθ)
    Ai = sparse([
        0 0 0 1 0 0;
        0 0 0 0 1 0;
        0 0 0 0 0 1;
        0 0 0 -bx / m 0 0;
        0 0 0 0 -by / m 0;
        0 0 0 0 0 -bθ / Izz
    ])
    
    Bi = sparse([
        0 0 0;
        0 0 0;
        0 0 0;
        1/m 0 0;
        0 1/m 0;
        0 0 1/Izz
    ])
    
    A = blockdiag(Ai, Ai)
    B = blockdiag(Bi, Bi)
    
    C = sparse(I(12))
    D = sparse(zeros(12, 6))
    
    return A, B, C, D
end;

In [150]:
function test_cluster_state_space()
    Izz = 1
    m = 10
    bx = 5
    by = 5
    bθ = 1
        
    A, B, C, D = cluster_state_space(m, Izz, bx, by, bθ)

    Aᵢ = sparse([
        0 0 0 1 0 0;
        0 0 0 0 1 0;
        0 0 0 0 0 1;
        0 0 0 -0.5 0 0;
        0 0 0 0 -0.5 0;
        0 0 0 0 0 -1
    ])
    
    Bᵢ = sparse([
        0 0 0
        0 0 0
        0 0 0
        0.1 0 0
        0 0.1 0
        0 0 1
    ])

    @test all(isapprox.(A, blockdiag(Aᵢ, Aᵢ), atol=1e-3))
    @test all(isapprox.(B, blockdiag(Bᵢ, Bᵢ), atol=1e-3))
    @test all(isapprox.(C, I(12), atol=1e-3))
    @test all(isapprox.(D, zeros(12,6), atol=1e-3))
end

test_cluster_state_space()

Test Passed